In [8]:
import pandas as pd
df = pd.read_csv('heart.csv')

print(df.columns)

Index(['id', 'age', 'sex', 'dataset', 'cp', 'trestbps', 'chol', 'fbs',
       'restecg', 'thalch', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype='object')


In [9]:
df = df[['age', 'sex', 'trestbps', 'chol', 'thalch', 'num']]
df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})


In [10]:

df = df[['age', 'sex', 'trestbps', 'chol', 'thalch', 'target']] 

In [11]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='target')
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [21]:
import numpy as np

class CustomGaussianNB:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.means = {}
        self.vars = {}
        self.priors = {}
        
        for c in self.classes:
            X_c = X[y == c]
            self.means[c] = X_c.mean()
            self.vars[c] = X_c.var()
            self.priors[c] = X_c.shape[0] / X.shape[0]

    def _gaussian_likelihood(self, x, mean, var):
        eps = 1e-9  
        coeff = 1 / np.sqrt(2 * np.pi * var + eps)
        exponent = np.exp(- (x - mean) ** 2 / (2 * var + eps))
        return coeff * exponent

    def predict(self, X):
        predictions = []
        for _, row in X.iterrows():
            posteriors = []
            for c in self.classes:
                prior = np.log(self.priors[c])
                likelihoods = self._gaussian_likelihood(row, self.means[c], self.vars[c])
                posterior = prior + np.sum(np.log(likelihoods))
                posteriors.append(posterior)
            predictions.append(self.classes[np.argmax(posteriors)])
        return np.array(predictions)


In [22]:
from sklearn.metrics import accuracy_score, classification_report

model = CustomGaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("🔹 Accuracy:", accuracy_score(y_test, y_pred))
print("\n🔹 Classification Report:\n", classification_report(y_test, y_pred))


🔹 Accuracy: 0.7391304347826086

🔹 Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.73      0.70        75
           1       0.80      0.74      0.77       109

    accuracy                           0.74       184
   macro avg       0.73      0.74      0.73       184
weighted avg       0.75      0.74      0.74       184

